In [22]:
import pandas as pd
import requests
import hashlib

# Set up API credentials
API_PUBLIC_KEY = "6edaa470d00da7b9dad29966b0243043"
API_PRIVATE_KEY = "ce64d5e2586b56d73e726c44dd3108bff15084f0"

# Set up timestamp parameter
timestamp = "1"

# Generate hash value
to_hash = timestamp + API_PRIVATE_KEY + API_PUBLIC_KEY
hash_value = hashlib.md5(to_hash.encode()).hexdigest()

# Set up request parameters, including the hash parameter
params = {
    "apikey": API_PUBLIC_KEY,
    "ts": timestamp,
    "hash": hash_value,
    "limit": 100  # Set the number of characters to retrieve per request
}

# Make API requests to retrieve all characters
all_characters = []

offset = 0
total_characters = -1

while total_characters == -1 or offset < total_characters:
    params["offset"] = offset
    response = requests.get(
        "https://gateway.marvel.com/v1/public/characters", params=params)

    if response.status_code == 200:
        data = response.json()["data"]
        total_characters = data["total"]

        # Extend the list with the characters from the current response
        all_characters.extend(data["results"])

        offset += params["limit"]
    else:
        print("Error retrieving characters:", response.status_code)
        break

# Create a DataFrame from the character data
df = pd.DataFrame(all_characters)

# Write DataFrame to CSV file
df.to_csv("marvel_characters.csv", index=False)
